# Машинное обучения для текстов

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


### Описание данных

Столбец *text* в содержит текст комментария, а *toxic* — целевой признак.

#### Часть 1. [Подготовка](#part1)
* [1. Импорт библиотек.](#part1.1)
* [2. Загрузка и подготовка данных](#part1.2)

#### Часть 2. [Обучение](#part2)
* [2.1 Решающее дерево](#part2.1)
* [2.2 Логистическая регрессия](#part2.2)
* [2.3 Метод опорных векторов (SVM)](#part2.3)

#### Часть 3. [Предсказание](#part3)

#### Часть 4. [Общий вывод](#part4)

<a id='part1'></a>
# 1. Подготовка
<a id='part1.1'></a>
## 1.1 Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
import spacy
import re
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import time

<a id='part1.2'></a>
## 2. Загрузка и подготовка данных

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()
#
corpus = data['text'].values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Создадим функцию для подготовки текста. Она удаляет из текста все символы, кроме букв, пробелов и апострофов. Далее, с помощью библиотеки Spacy, лемматизирует полученный текст и удаляет из лемм стоп-слова.

In [3]:
lemmatizer = English()
def prepare_text(text):
    # очистка
    cleared_text = " ".join(re.sub(r'[^a-zA-Z\' ]', ' ', text).split())
    # лемматизация
    lemmatized = [token.lemma_ for token in lemmatizer(cleared_text)]
    # стоп-слова
    filtered_and_lemmatized_list = []
    for word in lemmatized:
        lexeme = lemmatizer.vocab[word]
        if lexeme.is_stop == False:
            filtered_and_lemmatized_list.append(word)
    
   
    filtered_and_lemmatized = " ".join(filtered_and_lemmatized_list)
    
    return filtered_and_lemmatized

Проверка функции

In [4]:
print("Исходный текст:", corpus[0])
print("Очищенный и лемматизированный текст без стоп-слов:", prepare_text(corpus[0]))

Исходный текст: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный и лемматизированный текст без стоп-слов: Explanation edits username Hardcore Metallica Fan reverted vandalisms closure GAs voted New York Dolls FAC remove template talk page -PRON- retired


Применяем функцию на весь датасет

In [5]:
for i in range(len(corpus)):
    corpus[i] = prepare_text(corpus[i])

Разобьем датасет на тренировочную и тестовую выборки. Посчитаем для них TF-IDF.

In [6]:
X_train,X_test,y_train,y_test = train_test_split(corpus, data['toxic'], test_size=0.2, random_state=1)

count_tf_idf = TfidfVectorizer() 

tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_test = count_tf_idf.transform(X_test)
print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (127656, 147963)
Размер матрицы: (31915, 147963)


## Вывод

Данные загружены и подготовлены. Все комментарии лемматизированы, убраны лишние символы и стоп-слова.
Данные разбиты на тренировучную и тестовую выборки, посчитан TF-IDF.

<a id='part2'></a>
# 2. Обучение
<a id='part2.1'></a>
## 2.1 Решающее дерево

In [7]:
%%script false --no-raise-error
#пропуск выполнения ячейки
%%time
model_dt = DecisionTreeClassifier(random_state=1)
model_params = {
            'max_depth': [i for i in range(1,200)],
            'min_samples_split' : [i for i in range(2,200)],
            'min_samples_leaf' : [i for i in range(1,200)],
}

rs_dt = RandomizedSearchCV(model_dt, model_params, scoring='f1', cv=5,
                           n_jobs=-1,n_iter=100,random_state=1)

search_rs_dt = rs_dt.fit(tf_idf_train,y_train)

print('Лучшие гиперпараметры:')
print(search_rs_dt.best_estimator_)
print('Лучший F1:')
print('{:.3f}'.format(search_rs_dt.best_score_))

Couldn't find program: 'false'


In [8]:
model_dt = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=121, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=199,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

<a id='part2.2'></a>
## 2.2 Логистическая регрессия

In [9]:
%%script false --no-raise-error
#пропуск выполнения ячейки
%%time
model_lr = LogisticRegression(max_iter=1500, random_state=1)
model_params = {
            'C': [1, 10, 15, 20],
            'penalty':['none', 'l2'],
            'solver' : ['newton-cg', 'lbfgs', 'liblinear']
}

rs_lr = RandomizedSearchCV(model_lr, model_params, scoring='f1', cv=3,
                           n_jobs=-1)

search_rs_lr = rs_lr.fit(tf_idf_train,y_train)

print('Лучшие гиперпараметры:')
print(search_rs_lr.best_estimator_)
print('Лучший F1:')
print('{:.3f}'.format(search_rs_lr.best_score_))

Couldn't find program: 'false'


In [10]:
model_lr = LogisticRegression(C=15, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

<a id='part2.3'></a>
## 2.3 Метод опорных векторов (SVM)

In [11]:
%%script false --no-raise-error
#пропуск выполнения ячейки
model_svm = svm.SVC(random_state=1)
model_params = {
            'C': [0.01, 0.1, 1, 10, 100, 1000],
            'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma': [1, 0.1, 0.01, 0.001]
}

rs_svm = RandomizedSearchCV(model_svm, model_params, scoring='f1', cv=3,
                           n_jobs=-1, n_iter=10)

search_rs_svm = rs_svm.fit(tf_idf_train,y_train)
print('Лучшие гиперпараметры:')
print(search_rs_svm.best_estimator_)
print('Лучший F1:')
print('{:.3f}'.format(search_rs_svm.best_score_))

Couldn't find program: 'false'


In [12]:
model_svm = svm.LinearSVC(C=1, random_state=1)

## Вывод

Были подобраны оптимальные гиперпараметры для трех моделей.

||Модель решашающего дерева|Логистическая регрессия|Метод опорных векторов|
|-|-|-|-|
| F1 на тренировочных данных|0.734|0.756|0.759|

<a id='part3'></a>
# 3. Предсказания

In [13]:
for model in [model_dt,model_lr,model_svm]:
    time0 = time.time()
    model.fit(tf_idf_train, y_train)
    time1 = time.time()
    predicted = model.predict(tf_idf_test)
    time2 = time.time()
    
    print('Модель: ', str(model).split('(')[0])
    print('F1: {:.3f}'.format(f1_score(y_test,predicted)))
    print("Время обучения: %s секунд"%(time1 - time0))
    print("Время предсказания: {} секунд".format(time2 - time1))

Модель:  DecisionTreeClassifier
F1: 0.736
Время обучения: 33.36553359031677 секунд
Время предсказания: 0.024006128311157227 секунд
Модель:  LogisticRegression
F1: 0.782
Время обучения: 1.354809284210205 секунд
Время предсказания: 0.003000974655151367 секунд
Модель:  LinearSVC
F1: 0.788
Время обучения: 0.6851546764373779 секунд
Время предсказания: 0.002001047134399414 секунд


||Модель решашающего дерева|Логистическая регрессия|Метод опорных векторов|
|-|-|-|-|
| Время обучения, с |33.36|1.354|0.685|
| Время предсказания, с |0.024|0.003|0.002|
| F1 на тестовых данных |0.736|0.782|0.788|

<a id='part4'></a>
# 4. Общий вывод


В результате работы были изучены и подготовлены данные с пользовательскими описаниями товаров. Проведена лемматизация, убраны стоп-слова.

Даннные поделены на обучающую и тестовую выборки, для каждой вычислены значения TF-IDF.

Было обучено три модели (решающее дерево, логистическая регрессия, опорные векторы) и методом случайного поиска были подобраны оптимальные гипермапараметры.

Все обученные модели были поверены на тестовой выборке.

Требуемое значение F1 (>0.75) было достигнуто на двух моделях из трех.

Самым лучшим качеством (F1 = 0.788) обладает модель метода опорных векторов.